In [44]:
from Bio import SeqIO
from glob import glob
import os

In [45]:
import numpy
numpy.__version__

'1.22.3'

In [46]:
from mito.genotyping import *

## Load reference

In [47]:
# Path to reference genome
reference_path = '../../data/reference/mito_GRCh38_gimlet.fasta'

with open(reference_path, 'r'):
    reference_seq = next(SeqIO.parse(reference_path, "fasta"))
reference = pd.Series(list(reference_seq.seq))

## Load read matrices

In [48]:
# Path to allele count files
INPUT_PATH = '../../data/A1_06_changed_name/'


# Parameters
#DATA_NAME = 'A1_06'
#BASE_PATH = f'../../../data/20210712_{DATA_NAME}'
#INPUT_PATH = f'{BASE_PATH}/cells/'
#e_rate = 0.0778
e_rate = 0.0556
p_mutation = 1 / 500.0

In [49]:
cell_count = []
cell_name = []
locations = range(1, 16571+1)

# Get all cell counts files in the input path
cell_count_filenames = list(glob(os.path.join(INPUT_PATH, '*.txt')))

for filename in sorted(cell_count_filenames):
    name = os.path.basename(filename).split('.')[0]

    # Load one cell count file
    counts = pd.read_csv(filename, sep='\t')
    print(f'Reading {filename}, {counts.shape[0]} locations')
    # Some files are missing some locations
    counts = counts.set_index('POS').reindex(locations).reset_index()
    counts.name = name
    
    cell_count.append(counts)
    cell_name.append(name)

Reading ../../data/A1_06_changed_name/AACCATCGGCAACTACCACT_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/AACCATCGGCCACAATCCAC_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/AACCATCGGCCACTAACCGG_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/AACCATCGGCCCGTTCGTAT_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/AACCATCGGCGAAGCTAGCC_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/AACCATCGGCGTAACGTCTA_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/AACCATCGGCTCCAATGCAA_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/AACCATCGGCTTATGCGTTG_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/AACCATCGGCTTCAGGTTCC_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/AACCATCGGCTTGCCTTATC_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/AACCATCGGCTTGCGACCAC_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/AACGTGACAAACTACCGAGG_ac.txt, 16569 locations
Read

Reading ../../data/A1_06_changed_name/AGCTCCACTGTTCCTAGAAG_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/AGCTCCACTGTTCGAAGACG_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/AGCTCCACTGTTCTGTTACG_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/AGCTCCACTGTTGGCGTGTG_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/AGCTCCACTGTTGTGCTAGT_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/AGGTGATGCGCAAGATTCCA_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/AGGTGATGCGCCACTTGTCC_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/AGGTGATGCGCTAGGCATAT_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/AGGTGATGCGTAACGCTTAG_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/AGGTGATGCGTCAAGCGAGT_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/AGGTGATGCGTGGACGGAGT_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/AGGTGATGCGTTACCTCAGT_ac.txt, 16569 locations
Read

Reading ../../data/A1_06_changed_name/CACATCCAGGTCCTAACCAT_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/CACATCCAGGTGGACGGAGT_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/CACATCCAGGTTACCTCAGT_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/CACATCCAGGTTCTTCACCA_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/CACATCCAGGTTCTTGCGAT_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/CACATCCAGGTTCTTGTCCT_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/CACATCCAGGTTGAGGAAGA_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/CACATCCAGGTTGCGCTACG_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/CACATCCAGGTTGGAGAGGA_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/CACGGTGTATAACTAGCGTA_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/CACGGTGTATCAAGATTCCA_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/CACGGTGTATCCACTTGTCC_ac.txt, 16569 locations
Read

Reading ../../data/A1_06_changed_name/CCAATTCATGTAGTCCTTGA_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/CCAATTCATGTCGGTTCATT_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/CCAATTCATGTCTCACGAAT_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/CCAATTCATGTGATCGGCAG_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/CCAATTCATGTGGATGTCCG_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/CCAATTCATGTGGCTACTAA_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/CCAATTCATGTGTACGGTGC_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/CCAATTCATGTTAAGAGAGC_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/CCAATTCATGTTACGTGTCC_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/CCAATTCATGTTAGTGGTGC_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/CCAATTCATGTTCATTCGCA_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/CCAATTCATGTTCCGCTTCT_ac.txt, 16569 locations
Read

Reading ../../data/A1_06_changed_name/CCATCACTATCGACGGTTGT_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/CCATCACTATCGAGTCATTA_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/CCATCACTATGAAGCTAGCC_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/CCATCACTATGTAACGTCTA_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/CCATCACTATTCCAATGCAA_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/CCATCACTATTGCACTATCC_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/CCATCACTATTTACGCGCGA_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/CCATCACTATTTATGCGTTG_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/CCATCACTATTTCAGGTTCC_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/CCATCACTATTTGCCTTATC_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/CCATCACTATTTGCGACCAC_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/CCATTACAGTCAAGAGGTCT_ac.txt, 16569 locations
Read

...

Reading ../../data/A1_06_changed_name/TTCCGCTAGCCGACGGTTGT_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTCCGCTAGCGTAACGTCTA_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTCCGCTAGCTCCAATGCAA_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTCCGCTAGCTGCACTATCC_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTCCGCTAGCTTATGCGTTG_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTCCGCTAGCTTCAGGTTCC_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTCCGCTAGCTTGCCTTATC_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTCCGCTAGCTTGCGACCAC_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTCCTCTGTAAACTACCACT_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTCCTCTGTACACAATCCAC_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTCCTCTGTACACTAACCGG_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTCCTCTGTACCGTTCGTAT_ac.txt, 16569 locations
Read

Reading ../../data/A1_06_changed_name/TTCTACGATGGGAGATTGAA_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTCTACGATGTAACCGGTAA_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTCTACGATGTACTCAGGAG_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTCTACGATGTAGTCCTTGA_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTCTACGATGTCCAAGCGTG_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTCTACGATGTCGGTTCATT_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTCTACGATGTCTCACGAAT_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTCTACGATGTGATCGGCAG_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTCTACGATGTGCATATGGT_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTCTACGATGTGGATGGCAA_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTCTACGATGTGGATGTCCG_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTCTACGATGTGGCTACTAA_ac.txt, 16569 locations
Read

Reading ../../data/A1_06_changed_name/TTGCCGTCAATTGAGGAAGA_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTGCCGTCAATTGCGCTACG_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTGCCGTCAATTGGAGAGGA_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTGCCTCTAGAACTAGCGTA_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTGCCTCTAGCAAGATTCCA_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTGCCTCTAGCCACTTGTCC_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTGCCTCTAGCCGAATTCAC_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTGCCTCTAGCTAGGCATAT_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTGCCTCTAGTAACGCTTAG_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTGCCTCTAGTCAAGCGAGT_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTGCCTCTAGTGGACGGAGT_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTGCCTCTAGTTACCTCAGT_ac.txt, 16569 locations
Read

Reading ../../data/A1_06_changed_name/TTGCTTGGCCGAATGAACGT_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTGCTTGGCCTGACAGAGGT_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTGCTTGGCCTGCAGGCTAC_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTGCTTGGCCTTACCGGACC_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTGCTTGGCCTTATGCCGCT_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTGCTTGGCCTTATTCGCCT_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTGCTTGGCCTTCTCGCCTC_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTGGAGCAATACTACCGAGG_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTGGAGCAATCAAGAGAACG_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTGGAGCAATCCAACATCAT_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTGGAGCAATCCGAGAAGGA_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTGGAGCAATCCTATAATGG_ac.txt, 16569 locations
Read

Reading ../../data/A1_06_changed_name/TTGTGCCGAGCGGTTGGCTA_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTGTGCCGAGGAATGAACGT_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTGTGCCGAGGAGCGGAACT_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTGTGCCGAGTGACAGAGGT_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTGTGCCGAGTGCAGGCTAC_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTGTGCCGAGTTACCGGACC_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTGTGCCGAGTTATGCCGCT_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTGTGCCGAGTTATTCGCCT_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTGTGCCGAGTTCTCGCCTC_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTGTGCCGAGTTGCTAAGGA_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTGTGGTTCGACTACCGAGG_ac.txt, 16569 locations
Reading ../../data/A1_06_changed_name/TTGTGGTTCGCAAGAGAACG_ac.txt, 16569 locations
Read

In [50]:
def remove_low_cov_pos(cell_count):
    
    # Filter positions with low mean coverage
    
    # First make list with positions with low coverage
    # concat cell counts into new df
    cc_flt = pd.concat(cell_count, axis=1)
    # drop unwanted cols
    cc_flt.drop(['#CHR', 'POS', 'Count_A', 'Count_C', 'Count_G', 'Count_T'], axis=1, inplace=True)
    # drop bulk col
    column_numbers = [x for x in range(cc_flt.shape[1])] # list of columns' integer indices
    column_numbers.remove(0) #removing column integer index 0
    cc_flt = cc_flt.iloc[:, column_numbers] #return all columns except the 0th column
    
    # compute average coverage
    cc_flt['mean'] = cc_flt.mean(axis=1)
    
    # Get positions in new col
    cc_flt.reset_index(inplace=True)
    cc_flt['POS'] = cc_flt['index']+1
    
    # make new df with only relevant info
    cc_flt_pos = cc_flt[['mean', 'POS']]
    
    # reset index
    cc_flt_pos.set_index('POS', inplace=True)
    
    # Select rows with low coverage
    threshold = 100
    below_thres = cc_flt_pos.loc[cc_flt_pos['mean'] < threshold]

    # save index (which is POS) to list
    below_thres_lst = below_thres.index.tolist()
    
    return below_thres_lst

In [51]:
# Obtain a set of error rates to test


#e_rates = [0.0778]
e_rates = [0.0556]

#e_rates = np.linspace(0.0001, 0.0009, 10).round(4)
#extra_error_rates = [0.001, 0.002, 0.005]
#extra_error_rates_ar = np.array(extra_error_rates)
#all_e_rates = np.concatenate((e_rates1, extra_error_rates_ar))

all_e_rates = e_rates
all_e_rates

[0.0556]

In [52]:
#np.linspace(0.0001, 0.22, 10).round(4)

## Compute mutations probabilities

In [53]:
def cell_prob(e_rate):
    
    error_rate_when_no_mutation = error_rate_when_mutation = e_rate
    p_mutation = 1 / 500.0

    cell_prob = []
    for count in cell_count:
        count = count.iloc[:reference.shape[0]]  # discard trailing positions
        p = nucleotide_mutation_prob(
            cell_counts=count,
            reference=reference,
            error_rate_when_no_mutation=error_rate_when_no_mutation,
            error_rate_when_mutation=error_rate_when_mutation,
            p_mutation=p_mutation,
        )

        cell_prob.append(p)
        
    # Compute P(mutation | read counts)
    cells_p_mutation = []
    for cell_prob in cell_prob:
        p = mutation_prob(cell_prob, reference)
        cells_p_mutation.append(p)
        
        
    # Make mutation matrix
    mutation_matrix = cells_p_mutation[0][['#CHR', 'POS']].copy()
    mutation_matrix_data = pd.DataFrame(np.dstack([c['Prob_mutation'].values for c in cells_p_mutation]).squeeze())
    mutation_matrix = pd.concat([mutation_matrix, mutation_matrix_data], axis=1)
    
    return mutation_matrix      

In [54]:
mutation_matrix = cell_prob(0.0556)

/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is de

/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is de

/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is de

/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is de

/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is de

...

/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is de

/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is de

/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is de

/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is de

/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:253: FutureWarning: The 'lookup' method is de

In [42]:
mutation_matrix.shape

(16569, 3282)

In [41]:
mutation_matrix

,#CHR,POS,0,1,2,3,4,5,6,7,...,3270,3271,3272,3273,3274,3275,3276,3277,3278,3279
0,MT,1,0.000134,NaN,NaN,NaN,0.002000,NaN,0.000668,0.002,...,0.000501,0.002000,NaN,0.000223,0.000200,0.001001,0.000223,NaN,NaN,0.002
1,MT,2,0.000134,NaN,NaN,NaN,0.002000,NaN,0.000668,0.002,...,0.000401,0.002000,NaN,0.000223,0.000167,0.001001,0.000223,NaN,NaN,0.002
2,MT,3,0.000134,NaN,NaN,NaN,0.002000,NaN,0.000668,0.002,...,0.000401,0.002000,NaN,0.000223,0.000167,0.001001,0.000182,NaN,NaN,0.002
3,MT,4,0.000134,NaN,NaN,NaN,0.002000,NaN,0.000668,0.002,...,0.000401,0.002000,NaN,0.000223,0.000167,0.001001,0.000182,NaN,NaN,0.002
4,MT,5,0.000134,NaN,NaN,NaN,0.002000,NaN,0.000668,0.002,...,0.000401,0.002000,NaN,0.000223,0.000167,0.001001,0.000182,NaN,NaN,0.002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16564,MT,16565,NaN,0.002,NaN,NaN,0.002000,NaN,0.001001,NaN,...,NaN,0.001001,NaN,0.000668,0.000334,0.000401,0.000668,NaN,NaN,NaN
16565,MT,16566,NaN,0.002,NaN,NaN,0.002000,NaN,0.001001,NaN,...,NaN,0.001001,NaN,0.000668,0.000334,0.000401,0.000668,NaN,NaN,NaN
16566,MT,16567,NaN,0.002,NaN,NaN,0.001001,NaN,0.001001,NaN,...,NaN,0.001001,NaN,0.000501,0.000334,0.000401,0.000668,NaN,NaN,NaN
16567,MT,16568,NaN,0.002,NaN,NaN,0.001001,NaN,0.001001,NaN,...,NaN,0.001001,NaN,0.000501,0.000334,0.000401,0.000668,NaN,NaN,NaN


In [43]:
mutation_matrix.isna().sum().sum()

5839158

## Filter pmat

In [27]:
#def flt_pmat(e_rate, cell_count):
mutation_matrix = mutation_matrix
print('no filters' + str(mutation_matrix.shape))

# Remove germline mutations
#mutation_matrix = mutation_matrix.loc[mutation_matrix[0] < 0.9]

# Probabilities for location with mutation at any of the cells
mutation_threshold = 0.9
data = mutation_matrix.set_index('POS').drop('#CHR', axis=1)
mutation_any_cell = data[(data > mutation_threshold).any(axis=1)]

# select rows that have a value greater than mutation_threshold in more than x cells
mutation_threshold = 0.9
mask = (data.values > mutation_threshold).sum(axis=1) > 1 #create an array with boolean values and sum these along the axis and select rows with more than 1 True
two_cells_have_mut = data[mask]

# Define size of 50% of the population
half = len(two_cells_have_mut.columns) / 2

# Remove rows where 50% of cols are nan and save in new df
nan_rows = two_cells_have_mut.drop(two_cells_have_mut[(two_cells_have_mut.isna()).sum(axis=1)> half].index)
print('remove rows where 50% is nan' + str(nan_rows.shape))

# Replace NaN with row mean
imputed = nan_rows.transpose().fillna(nan_rows.mean(axis=1)).transpose()

# To skip removal of sites with overall high probability for mut
#high_prob_rows = imputed.copy()

# remove locations close to another location
index = imputed.index.tolist() + [2000000]
ind = [ a for a,b in zip(index[:-1], index[1:]) if b-a > 4]
clust = imputed.loc[ind]
print('remove cluster mutations' + str(clust.shape))

#clust.shape

# Replace 1.0
replaced = clust.replace(1.0, 0.99999)

# Remove pos with low cov
below_thres_lst = remove_low_cov_pos(cell_count)


low_dp = replaced.copy()
low_dp.reset_index(inplace = True)
low_dp = low_dp[~low_dp['POS'].isin(below_thres_lst)]
low_dp.set_index('POS', inplace=True)

print('remove positions with low dp' + str(low_dp.shape))

# save matrix
#np.set_printoptions(suppress=True)
#e_rate_name = str(e_rate)
#matrix_path = '../../data/A6_old_files' + e_rate_name + '.csv'
#low_dp.to_csv(matrix_path, index=False, sep=' ', header= False)
    
    #return low_dp
    


no filters(16569, 3282)
remove rows where 50% is nan(672, 3280)
remove cluster mutations(487, 3280)
remove positions with low dp(151, 3280)


In [57]:
#for e_rate in all_e_rates:
    #flt_pmat(e_rate, cell_count)